# Computing the Price Movements using Single Asset GBM models for Portfolio Asset Selection


Here we will build a function to generate a GBM model for the assets of interest

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
  Activating project at `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`


### Constants
Let's set some constant values that we will use below:

In [2]:
Δt = (1.0/252.0); # default timestep: 1-trading day in units of years
T = 48; # number of trading days for our projection
number_of_trading_days = 194; # number of trading days in the 2023 sample
number_of_sample_paths = 10000; # number of sample paths that we used to calculate to sample the model
all_range = range(1,stop=number_of_trading_days,step=1) |> collect; # range of possible time steps

### Setup the $\beta$-array
We'll simulate a range of perturbation values between a lower bound $\beta_{1}$ and an upper bound $\beta_{2}$, where we specify the number of test points between $\beta_{1}\rightarrow\beta_{2}$. We save this array in the `β` variable:

In [3]:
number_of_test_points = 100;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁,stop=β₂, length=number_of_test_points) |> collect;

In [4]:
dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

In [5]:
list_of_firms = keys(dataset) |> collect |> sort;

#### Load parameters dataset
This has been modified from PS2 to import relevant helper functions for estimating values for the drift and volatility parameters for each firm in our dataset. We load this data by calling the `MyFirmParametersDataSet()` function, and save these values in the `parameters` variable (which is type `DataFrame`):

In [6]:
function _loadcsvfile(path::String)::DataFrame
    return CSV.read(path, DataFrame);
end

_loadcsvfile (generic function with 1 method)

In [7]:
MyFirmParametersDataSet() = _loadcsvfile(joinpath(_PATH_TO_DATA, "Parameters-SP500-2018-2022.csv"));

In [8]:
parameters = MyFirmParametersDataSet()

Row,index,ticker,drift,volatility
,Int64,String7,Float64,Float64
1,1,MMM,-0.0822692,0.238729
2,2,AOS,0.0441975,0.266025
3,3,ABT,0.151149,0.230271
4,4,ABBV,0.118596,0.266743
5,6,ACN,0.178695,0.250005
6,7,ATVI,0.0745844,0.305086
7,8,ADM,0.161607,0.237929
8,9,ADBE,0.171392,0.307362
9,10,AAP,0.0890969,0.307218


In [9]:
function sample(model::MyGeometricBrownianMotionEquityModel, data::NamedTuple; 
    number_of_paths::Int64 = 100)::Array{Float64,2}

    # get information from data -
    T₁ = data[:T₁]
    T₂ = data[:T₂]
    Δt = data[:Δt]
    Sₒ = data[:Sₒ]

    # get information from model -
    μ = model.μ
    σ = model.σ

	# initialize -
	time_array = range(T₁, stop=T₂, step=Δt) |> collect
	number_of_time_steps = length(time_array)
    X = zeros(number_of_time_steps, number_of_paths + 1) # extra column for time -

    # put the time in the first col -
    for t ∈ 1:number_of_time_steps
        X[t,1] = time_array[t]
    end

	# replace first-row w/Sₒ -
	for p ∈ 1:number_of_paths
		X[1, p+1] = Sₒ
	end

	# build a noise array of Z(0,1)
	d = Normal(0,1)
	ZM = rand(d,number_of_time_steps, number_of_paths);

	# main simulation loop -
	for p ∈ 1:number_of_paths
		for t ∈ 1:number_of_time_steps-1
			X[t+1,p+1] = X[t,p+1]*exp((μ - σ^2/2)*Δt + σ*(sqrt(Δt))*ZM[t,p])
		end
	end

	# return -
	return X
end

function 𝔼(model::MyGeometricBrownianMotionEquityModel, data::NamedTuple)::Array{Float64,2}

    # get information from data -
    T₁ = data[:T₁]
    T₂ = data[:T₂]
    Δt = data[:Δt]
    Sₒ = data[:Sₒ]
    
    # get information from model -
    μ = model.μ

    # setup the time range -
    time_array = range(T₁,stop=T₂, step = Δt) |> collect
    N = length(time_array)

    # expectation -
    expectation_array = Array{Float64,2}(undef, N, 2)

    # main loop -
    for i ∈ 1:N

        # get the time value -
        h = (time_array[i] - time_array[1])

        # compute the expectation -
        value = Sₒ*exp(μ*h)

        # capture -
        expectation_array[i,1] = h + time_array[1]
        expectation_array[i,2] = value
    end
   
    # return -
    return expectation_array
end

Var(model::MyGeometricBrownianMotionEquityModel, data::NamedTuple) = _𝕍(model, data);
function _𝕍(model::MyGeometricBrownianMotionEquityModel, data::NamedTuple)::Array{Float64,2}

    # get information from data -
    T₁ = data[:T₁]
    T₂ = data[:T₂]
    Δt = data[:Δt]
    Sₒ = data[:Sₒ]

    # get information from model -
    μ = model.μ
    σ = model.σ

    # setup the time range -
    time_array = range(T₁,stop=T₂, step = Δt) |> collect
    N = length(time_array)

    # expectation -
    variance_array = Array{Float64,2}(undef, N, 2)

    # main loop -
    for i ∈ 1:N

        # get the time value -
        h = time_array[i] - time_array[1]

        # compute the expectation -
        value = (Sₒ^2)*exp(2*μ*h)*(exp((σ^2)*h) - 1)

        # capture -
        variance_array[i,1] = h + time_array[1]
        variance_array[i,2] = value
    end
   
    # return -
    return variance_array
end

_𝕍 (generic function with 1 method)

In [10]:
using Colors
using StatsPlots
# load colors -
colors = Dict{Int64,RGB}()
colors[1] = colorant"#EE7733";
colors[2] = colorant"#0077BB";
colors[3] = colorant"#33BBEE";
colors[4] = colorant"#EE3377";
colors[5] = colorant"#CC3311";
colors[6] = colorant"#009988";
colors[7] = colorant"#BBBBBB";

In [11]:
function generate_GBM_sim(ticker::String, dataset::Dict{String, DataFrame}, parameters::DataFrame)
    
    df = dataset[ticker]
    
    μ̂ = filter(:ticker=> x-> x == ticker, parameters) |> x-> x[1,:drift]
    σ̂ = filter(:ticker=> x-> x == ticker, parameters) |> x-> x[1,:volatility];

    start_index = rand(1:(number_of_trading_days - T - 1))
    stop_index = start_index + T

    model = build(MyGeometricBrownianMotionEquityModel, (
            μ = μ̂, σ = σ̂ ));

    Sₒ = df[start_index, :volume_weighted_average_price];
    T₁ = start_index*Δt
    T₂ = stop_index*Δt
    X = sample(model, (Sₒ = Sₒ, T₁ = T₁, T₂ = T₂, Δt = Δt), 
        number_of_paths = number_of_sample_paths);

    expectation = 𝔼(model, (Sₒ = Sₒ, T₁ = T₁, T₂ = T₂, Δt = Δt));
    variance = Var(model, (Sₒ = Sₒ, T₁ = T₁, T₂ = T₂, Δt = Δt));

    L68 = expectation[:,2] .- sqrt.(variance[:,2])
    U68 = expectation[:,2] .+ sqrt.(variance[:,2])
    L95 = expectation[:,2] .- 1.96*sqrt.(variance[:,2])
    U95 = expectation[:,2] .+ 1.96*sqrt.(variance[:,2])
    L99 = expectation[:,2] .- 2.576*sqrt.(variance[:,2])
    U99 = expectation[:,2] .+ 2.576*sqrt.(variance[:,2])

    p = plot(expectation[:,1], expectation[:,2], ribbon=(expectation[:,2]-L68, U68-expectation[:,2]), 
    fillalpha=0.5, fillcolor=:blue, label="68% CI")
    plot!(expectation[:,1], expectation[:,2], ribbon=(expectation[:,2]-L95, U95-expectation[:,2]), 
        fillalpha=0.3, fillcolor=:green, label="95% CI")
    
    plot!(expectation[:,1], expectation[:,2], ribbon=(expectation[:,2]-L99, U99-expectation[:,2]), 
        fillalpha=0.2, fillcolor=:orange, label="99% CI")
    
    plot!(expectation[:,1], expectation[:,2], c=:black, lw=3, ls=:dash, label="Expectation")
    
    plot!(X[:,1], df[start_index:stop_index, :volume_weighted_average_price], lw=4, c=:red, 
        label="Firm-$(ticker) actual")
    xlabel!("Time (years)", fontsize=18)
    ylabel!("Firm-$(ticker) VWAP (USD/share)", fontsize=18)

    savefig(p, joinpath(joinpath(pwd(), "figs"), "GeometricBrownianMotionSim-$(ticker)-CHEME-5660-Fall-2023.pdf"))

end

generate_GBM_sim (generic function with 1 method)

In [12]:
tech_portfolio = ["AMD", "AAPL", "MSFT", "IBM", "ADBE", "AMZN"]

6-element Vector{String}:
 "AMD"
 "AAPL"
 "MSFT"
 "IBM"
 "ADBE"
 "AMZN"

In [13]:
for ticker in tech_portfolio
    generate_GBM_sim(ticker, dataset, parameters)
end